# Tarea 2
## Gabriel Daniel Aguilar Luna
### _Facultad de Ingeniería, Universidad Nacional Autónoma de México_
### _Ciudad de México, México_
#### gabriel.aguilar@ingenieria.unam.edu
***

## Desarollo:

A partir del corpus seleccionado en la tarea anterior realizar un modelo del lenguaje neuronal con base en la arquitectura propuesta por Bengio (2003). El corpus ya debe estar preprocesado. S´ıganse los siguientes pasos:

In [39]:
#Bibliotecas
from nltk.corpus import brown
from collections import Counter
from random import sample, choice
from itertools import chain
from copy import deepcopy
import numpy as np
from scipy.special import softmax
from math import log

In [2]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

 ## Insertar símbolos de inicio y final de cadena.

In [3]:
BOS = '<BOS>'
EOS = '<EOS>'
bse = [[BOS]+sent+[EOS] for sent in brown.sents()]

In [4]:
# Esta funcion devuelve una cadena conformada por las palabras en la lista
# de strings sent_list. Despues de cada palabra agrega un espacio ' '
# para facilitar la visualizacion.
# sent_list debe ser una lista de strings.
def sentString(sent_list):
    return ''.join(w+' ' for w in sent_list)[:-1]

In [5]:
# Comprobamos que a las sentancias se les han agregado los
# simbolos de inicio BOS y final EOS.
sentString(sample(bse, 1)[0])

"<BOS> The operations of its other plant in Rochdale and Leesona's former operations in Manchester were transferred to a recently acquired plant in the adjoining town of Heywood . <EOS>"

## Obtener los bigramas que aparecen en el texto (indexar numéricamente).

In [6]:
# tokens es una lista de todas las cadenas de todas las sentencias en el corpus.
tokens = [w for w in chain(*[sent for sent in bse])]
# tipos es una lista de todas las palabras diferentes en el corpus
tipos = set(tokens)
# indice es un diccionario que asigna como valor un numero entero a cada tipo
# las cadenas del la lista tipos son las llaves del diccionario.
indice = dict(zip(tipos, range(len(tipos))))

In [7]:
print("Tokens:\t\t",len(tokens))
print("Tipos:\t\t",len(tipos))
print("Indice:\t\t",len(indice))
print("Indice[BOS]:\t",indice[BOS])
print("Indice[EOS]:\t",indice[EOS])

Tokens:		 1275872
Tipos:		 56059
Indice:		 56059
Indice[BOS]:	 13727
Indice[EOS]:	 25641


In [8]:
# La variable numSents es una lista de las sentencias del corpus representadas
# por los indices numericos de las cadenas que las conforman
numSents = deepcopy(bse)
for i in range(len(numSents)):
    for j in range(len(numSents[i])):
        numSents[i][j] = indice[numSents[i][j]]

In [9]:
# Se muestra la sentencia más corta:
min(numSents, key=len)

[13727, 34720, 25641]

Se crea una lista de bigramas numericos y otra con los bigramas string.

In [10]:
bigramas = list(chain(*[zip(sent,sent[1:]) for sent in numSents]))
bigramas_str = list(chain(*[zip(sent,sent[1:]) for sent in bse]))

In [11]:
for muestra in range(50,54):
    print(bigramas[muestra],'\t',bigramas_str[muestra])

(19759, 31980) 	 ('the', 'praise')
(31980, 138) 	 ('praise', 'and')
(138, 55181) 	 ('and', 'thanks')
(55181, 27391) 	 ('thanks', 'of')


In [90]:
len(bigramas)

1218532

##  Entrenar con los bigramas la red neuronal y obtener los valores para los hiperparámetros. Tomar de 100 unidades para la primera capa oculta (capa lineal) y 300 para la segunda capa oculta (capa con tanh).

Variables útiles:

In [18]:
N = len(indice)
d = 100
m = 300
eta = 0.1
iters = 100

In [80]:
# Esta funcion regresa el vector one hot para el indice dado
oneHots_array = np.identity((N), dtype='uint8')
def get_oneHot(indi):
    return oneHots_array[indi,:]

In [91]:
class red_neuronal:
    
    def __init__(self, N_, d_, m_, eta_=0.1, iters_=100):
        self.N = N_
        self.d = d_
        self.m = m_
        self.eta = eta_
        self.iters = iters_        
        # Array correspondiente a la matriz C
        self.C = np.random.rand(self.d,self.N)
        # Array correspondiente a la matriz W
        self.W = np.random.rand(self.m,self.d)
        # Array correspondiente a la matriz U
        self.U = np.random.rand(self.N,self.m)
    
    def forward(self,xi):
        self.Ci = np.dot(self.C,xi)
        self.hi = np.tanh(np.dot(self.W,self.Ci))
        self.a = np.dot(self.U,self.hi)
        return softmax(self.a)
    
    def train(self, bigrams):
        riesgo = 0
        for it in range(self.iters):
            for bi in bigrams:
                xi = get_oneHot(bi[0])
                p = self.forward(xi)
                riesgo -= log(p[bi[1]])
                #print(riesgo)
                p[bi[1]] -= 1
                self.d_out = p
                self.d_h = (1 - np.square(self.hi))*np.dot(self.U.T, self.d_out)
                self.d_c = np.dot(self.W.T, self.d_h)
                self.U = self.U - self.eta * np.dot(self.d_out[np.newaxis].T, self.hi[np.newaxis])
                self.W = self.W - self.eta * np.dot(self.d_h[np.newaxis].T, self.Ci[np.newaxis])
                self.C = self.C - self.eta * np.dot(self.d_c[np.newaxis].T, xi[np.newaxis])
            print(riesgo)
    
    def sentProba(self, sentencia):
        sentencia = sentencia.split(' ')
        for w in range(len(sentencia)):
            sentencia[w] = indice[sentencia[w]]
        bis = list(chain(*[zip(sent,sent[1:]) for sent in sentencia]))
        proba = 1
        for bi in bis:
            proba *= self.forward(bi[0])[bi[1]]
        return proba

In [92]:
red = red_neuronal(N,d,m,eta,1)

In [93]:
red.train(bigramas)

KeyboardInterrupt: 

In [ ]:
sent1 = '<BOS> Do not go gentle into that good night . <EOS>' # Dylan Thomas
sent2 = '<BOS> I am become death , the destroyer of worlds . <EOS>' # Robert Oppenheimer
sent3 = '<BOS> It was the best of times , it was the worst of times , it was the season of light , it was the season of darkness , it was the spring of hope, it was the winter of despair . <EOS>' # Charles Dickens
sent4 = '<BOS> Anything you can imagine you can make real . <EOS>' # Jules Verne
sent5 = '<BOS> Rage , rage against the dying of the light . <EOS>' # Dylan Thomas

In [ ]:
red.sentProba(sent1)

In [ ]:
red.sentProba(sent2)

In [ ]:
red.sentProba(sent3)

In [ ]:
red.sentProba(sent4)

In [ ]:
red.sentProba(sent5)

***

## Fuentes de Consulta:

- [1] Numpy, "NumPy Reference — NumPy v1.21 Manual", Numpy.org, 2021. [Online]. Available: https://numpy.org/doc/stable/reference/index.html. [Accessed: 09- Aug- 2021].